In [32]:
import warnings
warnings.filterwarnings('ignore')
import random
import pandas as pd

In [1]:
# Preparing the concated DATASET For Label Training

In [ ]:
final_label_test_df = pd.DataFrame(cols)

import pandas as pd

train_df = pd.read_csv("train_labels_evd_concated.csv")

train_df.head()

train_df['len'] = train_df.evidence.apply(lambda x : len(x.split()))

mask = train_df['len'] < 230

df = train_df[mask]

train_df.shape

df.label.value_counts()

df["label"] = df.label.apply(lambda x: 1 if x=="SUPPORTS" else 0)

df.label.value_counts()

sup = df[df["label"]==1].sample(25000,random_state=123)
ref = df[df["label"]==0].sample(25000,random_state=12)
df_1 = pd.concat([sup,ref])

X = df[["claim","evidence"]]
y = df["label"]

 Below code is only for reducing the training dataset size for faster training 

X.shape

from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X, y,
                                        stratify=y, 
                                        test_size=0.40)

X.shape

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.2)

X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train,
                                                    stratify=y_train, 
                                                    test_size=0.22)

train = pd.DataFrame()
train[["claim","evidence"]] = X_train
train["label"] = y_train
dev = pd.DataFrame()
dev[["claim","evidence"]] = X_dev
dev["label"] = y_dev
test = pd.DataFrame()
test[["claim","evidence"]] = X_test
test["label"] = y_test

In [17]:
test.to_csv('./bert/lb_data/test.csv', index=False, header=True)

In [18]:
train = train.reset_index()
dev = dev.reset_index()
test = test.reset_index()

In [19]:
columns = ['index', 'claim', 'evidence','label']
rename = {'label':"label", "claim":"sentence1","evidence":"sentence2"}

In [20]:
train = train[columns]
dev = dev[columns]
test = test[["index",'claim', 'evidence']]

In [21]:
train.rename(columns=rename,inplace=True)
dev.rename(columns=rename,inplace=True)
test.rename(columns=rename,inplace=True)

In [22]:
train.to_csv('./bert/lb_data/train.tsv', sep='\t', index=False, header=True)
dev.to_csv('./bert/lb_data/dev.tsv', sep='\t', index=False, header=True)
test.to_csv('./bert/lb_data/test.tsv', sep='\t', index=False, header=True)

In [23]:
test.head()

,index,sentence1,sentence2
0,19799,Vincente Minnelli is a director.,In addition to having directed some of the mos...
1,108998,Led Zeppelin released an eponymous debut album...,Led Zeppelin is the eponymous debut studio alb...
2,84949,Gerard Lee directs.,Gerard Lee -LRB- born 1951 in Melbourne -RRB- ...
3,14019,Just My Luck (2006 film) features an American ...,"Following Just My Luck , Lohan focused on smal..."
4,31112,Odin travels independently only.,He is often accompanied by his animal companio...


In [31]:
train.head()

,index,sentence1,sentence2,label
0,60981,Snow White and the Seven Dwarfs (1937 film) is...,"It was a critical and commercial success , and...",0
1,60976,Doug Jones starred in a 2017 science fiction h...,"He has appeared in films such as Tank Girl , H...",1
2,36040,Slash exclusively played country music.,Velvet Revolver was an American hard rock supe...,0
3,80297,"Tommy Chong was born on May 23rd, 1938.","Thomas B. Kin Chong -LRB- born May 24 , 1938 -...",0
4,84695,Johan Heldenbergh starred in The Zookeepers Wife.,He gained international fame by starring in fi...,1


In [25]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import numpy as np

In [26]:
test_o = pd.read_csv('./bert/lb_data/test.csv')

In [27]:
test_o.head()

,claim,evidence,label
0,Vincente Minnelli is a director.,In addition to having directed some of the mos...,1
1,Led Zeppelin released an eponymous debut album...,Led Zeppelin is the eponymous debut studio alb...,0
2,Gerard Lee directs.,Gerard Lee -LRB- born 1951 in Melbourne -RRB- ...,1
3,Just My Luck (2006 film) features an American ...,"Following Just My Luck , Lohan focused on smal...",1
4,Odin travels independently only.,He is often accompanied by his animal companio...,0


In [28]:
np.array(test_o.label)

array([1, 0, 1, ..., 1, 1, 1])

In [29]:
test_pred = pd.read_csv('./bert/lb_bert_output/test_results.tsv', sep='\t',header=None)

In [30]:
y_pred = np.argmax(np.array(test_pred[[0,1]]), axis=1)

In [8]:
print('accuracy: {}'.format(accuracy_score(np.array(test_o.label), y_pred)))
print(classification_report(np.array(test_o.label), y_pred))

accuracy: 0.9378151260504202
              precision    recall  f1-score   support

           0       0.89      0.88      0.88      2580
           1       0.96      0.96      0.96      6940

    accuracy                           0.94      9520
   macro avg       0.92      0.92      0.92      9520
weighted avg       0.94      0.94      0.94      9520



In [50]:
# actual test

In [ ]:
file = 'test-unlabelled.json'
with open(file) as train_file:
    test = json.load(train_file)

In [ ]:
df = pd.read_csv("test_pred_top13.csv",index=False)

In [ ]:
cols = []
for key, item in tqdm(test.items()):
    evd_lst = df[df.claim_id == int(key)].evidence.to_list()
    if len(evd_lst) > 5:
        evd_lst = evd_lst[:4]
    if evd_lst:
        d = {}
        d["claim_id"] = key
        d["claim"] = item["claim"]
        evd_txt = " ".join(evd_lst)
        d["evidence"] = evd_txt
        cols.append(d)

# This is Actual Test Part 
run from here to get predictions

In [60]:
test_df = pd.read_csv('final_test_df_top13.csv')

In [61]:
test_df.shape

(9463, 3)

In [62]:
test_df["words_len"] = test_df["evidence"].apply(lambda x : len(x.split()))

mask2 = (test_df["words_len"] <= 250)
test_df = test_df[mask2]

In [63]:
test_df.shape

(9463, 4)

In [64]:
test_df["evidence"] = test_df["evidence"].apply(lambda x: x.replace("\n","").strip())

In [65]:
test_df.columns

Index(['claim', 'claim_id', 'evidence', 'words_len'], dtype='object')

In [44]:
test_df = test_df[['claim_id', 'claim', 'evidence']].rename(columns={"claim_id":"index"})

In [45]:
test_df.to_csv('./bert/data/test.csv', index=False, header=True)

In [46]:
test_df.shape

(9463, 3)

In [47]:
test_df.to_csv('./bert/data/test.tsv', sep='\t', index=False, header=True)

In [48]:
test_df.shape

(9463, 3)

In [49]:
test_df.head()

,index,claim,evidence
0,110000,Raven-Symoné is an Anglican.,Raven-Symoné Christina Pearman -LRB- -LSB- ˈre...
1,16079,Solanum contains plants with ornamental flowers.,It also contains the nightshades and horse net...
2,126871,The Icelandic Coast Guard is also known as Gae...,Iceland 's own defense of its territorial wate...
3,226458,Chadwick Boseman portrayed a film character.,Chadwick Aaron Boseman -LRB- born November 29 ...
4,51263,James Brolin is an orthodontist.,Josh James Brolin -LRB- -LSB- ˈbroʊlᵻn -RSB- b...


In [31]:
mr.Quality.value_counts()

1    2394
0    1155
Name: Quality, dtype: int64

In [50]:
df = pd.read_csv('./bert/data/test.csv')

In [51]:
df.shape

(9463, 3)

In [66]:
test_o = pd.read_csv('./bert/data/test.csv')

test_pred = pd.read_csv('./bert/lb_bert_output/test_results.tsv', sep='\t',header=None)

y_pred = np.argmax(np.array(test_pred[[0,1]]), axis=1)

In [67]:
test_o["label"] = y_pred

In [68]:
test_o['label'] = test_o.label.apply(lambda x: "SUPPORTS" if x==1 else "REFUTES")

In [69]:
test_o["index"].nunique()

9463

In [70]:
test_o.head()

,index,claim,evidence,label
0,110000,Raven-Symoné is an Anglican.,Raven-Symoné Christina Pearman -LRB- -LSB- ˈre...,REFUTES
1,16079,Solanum contains plants with ornamental flowers.,It also contains the nightshades and horse net...,SUPPORTS
2,126871,The Icelandic Coast Guard is also known as Gae...,Iceland 's own defense of its territorial wate...,SUPPORTS
3,226458,Chadwick Boseman portrayed a film character.,Chadwick Aaron Boseman -LRB- born November 29 ...,SUPPORTS
4,51263,James Brolin is an orthodontist.,Josh James Brolin -LRB- -LSB- ˈbroʊlᵻn -RSB- b...,REFUTES


In [71]:
test_o.label.value_counts()

SUPPORTS    4962
REFUTES     4501
Name: label, dtype: int64

In [72]:
4962+4501

9463

In [73]:
test_o.to_csv("test_pred_final_labels.csv",index=False)